# Notebook 4: Mountain Wave Test Case

This notebook will go through an example of nonhydrostatic flow over a mountain. This test case is outlined in the paper avaliable at this link: https://doi.org/10.1002/qj.603. 

The new features in this example are related to the introduction of the mountain. This requires that we deform the initial extruded mesh so that the bottom of the domain follows the specified mountain profile. We then have the issue that when we calculate the initial balanced conditions, we no longer have a flat bottom on which to specify the bottom boundary condition. We solve this problem by applying the boundary condition at the top and then applying a shift so that the bottom boundary condition is as required. The final new feature is the sponge layer which is required at the top of the domain to damp vertically propagating waves so that they are not reflected back into the domain by the nonphysical impermeable upper boundary.

We begin by importing the required functions from Firedrake and Gusto:

In [1]:
from gusto import *
from firedrake import (as_vector, VectorFunctionSpace,
                       PeriodicIntervalMesh, ExtrudedMesh, SpatialCoordinate,
                       exp, pi, cos, Function, conditional, Mesh, op2)

Define the time-step size and the duration of the simulation. 

In [2]:
dt = 5.0
tmax = 9000.

Like with the gravity wave example, we will create a periodic interval mesh in the horizontal and extrude this in the vertical dimension to create a two dimensional mesh.

In [3]:
nlayers = 70  # horizontal layers
columns = 180  # number of columns
L = 144000.
m = PeriodicIntervalMesh(columns, L)

# build volume mesh
H = 35000.  # Height position of the model top
ext_mesh = ExtrudedMesh(m, layers=nlayers, layer_height=H/nlayers)
Vc = VectorFunctionSpace(ext_mesh, "DG", 2)
coord = SpatialCoordinate(ext_mesh)
x = Function(Vc).interpolate(as_vector([coord[0], coord[1]]))
a = 1000.
xc = L/2.
x, z = SpatialCoordinate(ext_mesh)
hm = 1.
zs = hm*a**2/((x-xc)**2 + a**2)

UFL:WARNING Discontinuous Lagrange element requested on interval * interval, creating DQ element.


Now we transform the mesh to follow the defined terrain. A smoothing is applied in the vertical direction.

In [4]:
dirname = 'non_hydro_mountain_smootherz'

zh = 5000.
xexpr = as_vector([x, conditional(z < zh, z + cos(0.5*pi*z/zh)**6*zs, z)])

new_coords = Function(Vc).interpolate(xexpr)
mesh = Mesh(new_coords)

We will record the full velocity fields, as well as the perturbations to the potential temperature and density.

In [5]:
output = OutputParameters(dirname=dirname,
                          dumpfreq=int(tmax / (9*dt)),
                          dumplist=['u'],
                          perturbation_fields=['theta', 'rho'])

Set up the corresponding state:

In [6]:
parameters = CompressibleParameters(g=9.80665, cp=1004.)
diagnostic_fields = [CourantNumber(), VelocityZ()]

state = State(mesh,
              dt=dt,
              output=output,
              parameters=parameters,
              diagnostic_fields=diagnostic_fields)


For this model, we require the use of a sponge layer. Gusto can formulate the `sponge` with an inbuilt function.

In [7]:
sponge = SpongeLayerParameters(H=H, z_level=H-10000, mubar=0.15/dt)

We again will use the compressible Euler equations, but this time with the sponge layer applied. We will also give the specific function space for the velocity, density, and potential temperature.

In [8]:
eqns = CompressibleEulerEquations(state, "CG", 1, sponge=sponge)

# Initial conditions
u0 = state.fields("u")
rho0 = state.fields("rho")
theta0 = state.fields("theta")

# spaces
Vu = state.spaces("HDiv")
Vt = state.spaces("theta")
Vr = state.spaces("DG")

UFL:WARNING Discontinuous Lagrange element requested on interval * interval, creating DQ element.


Set up the relevant parameters for the test case. Most of these relate to thermodynamic conditions.

In [9]:
# Thermodynamic constants required for setting initial conditions
# and reference profiles
g = parameters.g
N = parameters.N
p_0 = parameters.p_0
c_p = parameters.cp
R_d = parameters.R_d
kappa = parameters.kappa

# N^2 = (g/theta)dtheta/dz => dtheta/dz = theta N^2g => theta=theta_0exp(N^2gz)
x, z = SpatialCoordinate(mesh)
Tsurf = 300.
thetab = Tsurf*exp(N**2*z/g)
theta_b = Function(Vt).interpolate(thetab)

Determine the Exner pressure using the hydrostatic balance function. (Is this because we start with a hydrostatic balance, then allow the simulation to move away from this?)

In [10]:
# Calculate hydrostatic exner
exner = Function(Vr)
rho_b = Function(Vr)

exner_params = {'ksp_type': 'gmres',
                'ksp_monitor_true_residual': None,
                'pc_type': 'python',
                'mat_type': 'matfree',
                'pc_python_type': 'gusto.VerticalHybridizationPC',
                # Vertical trace system is only coupled vertically in columns
                # block ILU is a direct solver!
                'vert_hybridization': {'ksp_type': 'preonly',
                                       'pc_type': 'bjacobi',
                                       'sub_pc_type': 'ilu'}}

compressible_hydrostatic_balance(state, theta_b, rho_b, exner,
                                 top=True, exner_boundary=0.5,
                                 params=exner_params)

    Residual norms for exner_solver_ solve.
    0 KSP preconditioned resid norm 2.222764887545e+02 true resid norm 2.069609815553e+06 ||r(i)||/||b|| 1.000000000000e+00
    1 KSP preconditioned resid norm 2.439348632851e-10 true resid norm 5.428050688591e-08 ||r(i)||/||b|| 2.622741082787e-14


More hydrostatic balance set up for initial conditions

In [11]:
def minimum(f):
    fmin = op2.Global(1, [1000], dtype=float)
    op2.par_loop(op2.Kernel("""
static void minify(double *a, double *b) {
    a[0] = a[0] > fabs(b[0]) ? fabs(b[0]) : a[0];
}
""", "minify"), f.dof_dset.set, fmin(op2.MIN), f.dat(op2.READ))
    return fmin.data[0]


p0 = minimum(exner)
compressible_hydrostatic_balance(state, theta_b, rho_b, exner,
                                 top=True, params=exner_params)
p1 = minimum(exner)
alpha = 2.*(p1-p0)
beta = p1-alpha
exner_top = (1.-beta)/alpha
compressible_hydrostatic_balance(state, theta_b, rho_b, exner,
                                 top=True, exner_boundary=exner_top, solve_for_rho=True,
                                 params=exner_params)


    Residual norms for exner_solver_ solve.
    0 KSP preconditioned resid norm 3.315484606629e+02 true resid norm 4.100966222339e+06 ||r(i)||/||b|| 1.000000000000e+00
    1 KSP preconditioned resid norm 3.627206392679e-10 true resid norm 1.327793588507e-07 ||r(i)||/||b|| 3.237757924643e-14
    Residual norms for exner_solver_ solve.
    0 KSP preconditioned resid norm 3.310172303592e+02 true resid norm 4.091155332443e+06 ||r(i)||/||b|| 1.000000000000e+00
    1 KSP preconditioned resid norm 3.627206384977e-10 true resid norm 9.769587860125e-08 ||r(i)||/||b|| 2.387977738868e-14
    Residual norms for rhosolver_ solve.
    0 KSP preconditioned resid norm 8.093018628356e-05 true resid norm 7.079661062612e+00 ||r(i)||/||b|| 1.000000000000e+00
    1 KSP preconditioned resid norm 2.454895163496e-17 true resid norm 9.134553638225e-15 ||r(i)||/||b|| 1.290252959490e-15


Project the initial conditions, using the balanced state we just calculated.

In [12]:
theta0.assign(theta_b)
rho0.assign(rho_b)
u0.project(as_vector([10.0, 0.0]))
remove_initial_w(u0)

state.set_reference_profiles([('rho', rho_b),
                              ('theta', theta_b)])

Set up the time-stepper for each transported variable.

In [13]:
transported_fields = [ImplicitMidpoint(state, "u"),
                      SSPRK3(state, "rho"),
                      SSPRK3(state, "theta", options=SUPGOptions())]

Set up the linear solver and time-stepper.

In [14]:
# Set up linear solver
linear_solver = CompressibleSolver(state, eqns)

# build time stepper
stepper = SemiImplicitQuasiNewton(eqns, state, transported_fields, linear_solver=linear_solver)

Now we would run the simulation! Note, that it does take a while ... . 

In [ ]:
# Here's what you would do:
# stepper.run(t=0, tmax=tmax)